# project/
# ├── main.py                 → experiment loop entry point
# ├── dataloader.py            → data preprocessing, loader, scaling
# ├── models.py               → HybridODE, LSTMHybrid
# ├── train.py                → training + early stopping
# ├── visualize.py            → metrics, plots
# └── config.py               → param_grid + experiment combos

# main.py

In [7]:
import os
import json
import csv
import torch
import random
import numpy as np
from itertools import product
from config import param_grid
from dataloader import SEIRDDataLoader, get_split_strategy
from models import HybridODE
from train import train_model
from visualize import visualize_supervised, visualize_trajectory

EXPERIMENT_LOG_PATH = "experiment_log.csv"

In [8]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [9]:
def log_experiment(run_id, config, exp_folder):
    log_exists = os.path.exists(EXPERIMENT_LOG_PATH)
    with open(EXPERIMENT_LOG_PATH, mode='a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["run_id", "folder"] + list(config.keys()))
        if not log_exists:
            writer.writeheader()
        writer.writerow({"run_id": run_id, "folder": exp_folder, **config})

In [10]:
def train_single_model(config, dataloader, train_loader, val_loader, folder_name):
    save_dir = os.path.join("results", folder_name)
    os.makedirs(save_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = HybridODE(
        input_dim=dataloader.num_features,
        hidden_dim=64,
        num_layers=2,
        output_dim=dataloader.num_features,
        solver=config["solver"],
        sensitivity=config["sensitivity"]
    ).to(device)

    trained_model, test_loader, _ = train_model({
        "dataset_path": "../../SEIR_CSV.csv",
        "sequence_length": config["sequence_length"],
        "scaler_path": os.path.join(save_dir, "scaler.pkl"),
        "split_strategy": config["split_strategy"],
        "batch_size": config["batch_size"],
        "learning_rate": config["learning_rate"],
        "hidden_dim": 64,
        "num_layers": 2,
        "solver": config["solver"],
        "sensitivity": config["sensitivity"],
        "epochs": config["epochs"],
        "test_size": 0.2,
        "val_size": 0.2,
        "save_dir": save_dir
    })

    visualize_supervised(
        trained_model,
        test_loader,
        scaler_path=os.path.join(save_dir, "scaler.pkl"),
        output_dir=save_dir
    )
    
    visualize_trajectory(
        trained_model,
        test_loader,
        scaler_path=os.path.join(save_dir, "scaler.pkl"),
        output_dir=os.path.join(save_dir, "trajectory_plots")
    )

In [11]:
def run_experiment(config, run_id):
    set_seed(config.get("seed", 42))  # default to 42 if not specified
    exp_folder = f"exp_{run_id:03d}"
    save_dir = os.path.join("results", exp_folder)
    os.makedirs(save_dir, exist_ok=True)

    print("\n==============================")
    print(f"Running {exp_folder}:")
    print(config)
    print("==============================")

    # Log experiment config
    log_experiment(run_id, config, exp_folder)
    with open(os.path.join(save_dir, "config.json"), "w") as f:
        json.dump(config, f, indent=2)

    # Prepare data
    dataloader = SEIRDDataLoader(
        dataset_path='../../SEIR_CSV.csv',
        sequence_length=config['sequence_length'],
        scaler_path=os.path.join(save_dir, 'scaler.pkl')
    )
    data = dataloader.data

    # Get loaders based on split strategy
    split_result = get_split_strategy(
        data, strategy=config["split_strategy"],
        sequence_length=config["sequence_length"],
        batch_size=config["batch_size"],
        val_size=0.2,
        test_size=0.2,
        window_size=config.get("window_size", 200),
        horizon=config.get("horizon", 25),
        stride=config.get("stride", 25)
    )

    # Handle multi-fold splits
    if isinstance(split_result, list):
        for fold, (train_loader, val_loader, _) in enumerate(split_result):
            print(f"→ Fold {fold + 1}/{len(split_result)}")
            fold_suffix = f"_fold{fold}"
            train_single_model(config, dataloader, train_loader, val_loader, f"{exp_folder}{fold_suffix}")
    else:
        train_loader, val_loader, test_loader = split_result
        train_single_model(config, dataloader, train_loader, val_loader, exp_folder)

In [12]:
if __name__ == "__main__":
    print(f"🚀 Using device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    print("CUDA available:", torch.cuda.is_available())
    print("Device count:", torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")
    combinations = list(product(*param_grid.values()))
    for run_id, combo in enumerate(combinations):
        config = dict(zip(param_grid.keys(), combo))
        run_experiment(config, run_id)

🚀 Using device: NVIDIA GeForce RTX 4080 SUPER
CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 4080 SUPER

Running exp_000:
{'split_strategy': 'train_val_test', 'solver': 'tsit5', 'sensitivity': 'adjoint', 'learning_rate': 0.01, 'batch_size': 10, 'window_size': 200, 'horizon': 25, 'stride': 25, 'sequence_length': 800, 'epochs': 2, 'seed': 42}
✅ Scaler loaded successfully!
✅ Scaler saved successfully!
Training with: cuda
✅ Scaler loaded successfully!
✅ Scaler saved successfully!
Epoch 1, Train Loss: 0.159292
Epoch 1, Validation Loss: 0.022331
Epoch 2, Train Loss: 0.013539
Epoch 2, Validation Loss: 0.002847
✅ Best model saved to results/exp_000/best_model.pth
📊 Training log saved to results/exp_000/training_log.csv
  Feature           MSE         RMSE          MAE  R2 Score
0       S  3.471269e+06  1863.134155  1523.373779  0.179093
1       E  4.972750e+04   222.996643   177.451782 -0.581595
2     Ins  2.769577e+03    52.626770    42.198692 -0.549595
3      Is  3.4876

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 25 and the array at index 1 has size 15